All parameters should be defined here

In [5]:
parameters = {
    "input_path" : "wallpaper.jpg",
    "output_path" : "output_image.jpg",
    "wave_dec_1" : {"wave_type" : 'haar', "level" : 3},
    "wave_dec_2" : {"wave_type" : 'db2', "level" : 5},
    "thresholds_percentage" : [0.05, 0.1, 0.15, 0.2]
}

Pick an image of your choice. Load the image using appropriate commands. If it is a colored image, convert it to grayscale. Save the image in a two-dimensional matrix X.

In [6]:
from PIL import Image
import numpy as np


# Define the input and output paths
input_path = parameters["input_path"]
output_path = parameters["output_path"]

# Load the input image
img = Image.open(input_path)


# Check if the image is grayscale
if img.mode != "L":
    # Convert the image to grayscale
    img = img.convert("L")

X = np.array(img)

Determine the total number of bytes needed to store this image in X

In [7]:
# Method 1
unique_items = np.unique(X) # Removing repeated items
bits_for_item = int(np.ceil(np.log2(len(unique_items)))) 
total_bytes = (X.size * bits_for_item) // 8
print("total number of bytes needed to store this image in X using Method 1 =", total_bytes)

# Method 2
total_bytes = X.nbytes # This function do the same as Method 1. so we will use it instead of Method 1
print("total number of bytes needed to store this image in X using Method 2 =", total_bytes)

total number of bytes needed to store this image in X using Method 1 = 14075528
total number of bytes needed to store this image in X using Method 2 = 14075528


Convert X to a vector-form. Call it x. Verify that the size (bytes) of the new variable x
matches that of X.

In [8]:
x = X.flatten() # Convert to 1d array

# this will raise an error statment if (x.nbytes != X.nbytes)
assert x.nbytes == X.nbytes, "the size (bytes) of the new variable x doesn't matches that of X"

Perform multi-level decomposition of this signal using two different wavelets of your own
choice:
• Perform 3-level decomposition
• Perform 5-level decomposition

In [9]:
import pywt

# Doing wavelet decomposition
wav_coff_1 = pywt.wavedec(x, parameters["wave_dec_1"]["wave_type"], level=parameters["wave_dec_1"]["level"])
wav_coff_2 = pywt.wavedec(x, parameters["wave_dec_2"]["wave_type"], level=parameters["wave_dec_2"]["level"])

In [10]:
print("Sizes after decomposition and before thresholding:\n")
size = sum([arr.nbytes for arr in wav_coff_1])
print("Signal 1 ==>", size, "bytes")
size = sum([arr.nbytes for arr in wav_coff_2])
print("Signal 2 ==>", size, "bytes")

Sizes after decomposition and before thresholding:

Signal 1 ==> 112604224 bytes
Signal 2 ==> 112604328 bytes


Perform hard thresholding on the decompositions from the previous step (i.e., the wavelet
coefficients resulting from the previous step). Perform the thresholding separately on both
3-level and 5-level decompositions. In this step, use a range of thresholds selected between
5%−20% of the maximum wavelet coefficient value. Specifically, use the following thresholds:
5%, 10%, 15%, 20%.

In [11]:
##############################################################################################
###### THIS CODE WAS CREATED FOR ONLY 1 SIGNAL (JUST FOR EXPLANATION) ==> DON'T RUN IT  ######
##############################################################################################

thresholds_percentage = parameters['thresholds_percentage']
thresholds = []
wav_coff_thresh = []

# Find the threshold for each array in the list
for i, arr in enumerate(wav_coff_1):
    max_value = np.max(arr)
    thresholds.append([])  # Create an empty list for each array
    for j in range(len(thresholds_percentage)):
      threshold = max_value * thresholds_percentage[j] 
      thresholds[i].append(threshold)

# Perform thresholding for each array individually
for arr, threshold_list in zip(wav_coff_1, thresholds):
    thresholded_arr = []
    for threshold in threshold_list:
        thresholded_arr.append(pywt.threshold(arr, threshold, 'hard'))
    wav_coff_thresh.append(thresholded_arr)

In [12]:
thresholds_percentage = [0.05, 0.1, 0.15, 0.2]
thresholds_1 = []
wav_coff_thresh_1 = []
thresholds_2 = []
wav_coff_thresh_2 = []

# Find the threshold for each array in the list
for i, (arr_1, arr_2) in enumerate(zip(wav_coff_1, wav_coff_2)):
    max_value_1 = np.max(arr_1)
    max_value_2 = np.max(arr_2)
    thresholds_1.append([])  # Create an empty list for each array
    thresholds_2.append([])  # Create an empty list for each array
    for j in range(len(thresholds_percentage)):
      threshold_1 = max_value_1 * thresholds_percentage[j] 
      thresholds_1[i].append(threshold_1)
      threshold_2 = max_value_2 * thresholds_percentage[j] 
      thresholds_2[i].append(threshold_2)

# Perform thresholding for each array individually
for arr_1, threshold_list_1, arr_2, threshold_list_2 in zip(wav_coff_1, thresholds_1, wav_coff_2, thresholds_2):
    thresholded_arr_1 = []
    thresholded_arr_2 = []
    for threshold_1, threshold_2 in zip(threshold_list_1, threshold_list_2):
        thresholded_arr_1.append(pywt.threshold(arr_1, threshold_1, 'hard'))
        thresholded_arr_2.append(pywt.threshold(arr_2, threshold_2, 'hard'))
    wav_coff_thresh_1.append(thresholded_arr_1)
    wav_coff_thresh_2.append(thresholded_arr_2)

In [13]:
old_sizes_1 = []
old_sizes_2 = []

print("Sizes after thresholding and before quantization:\n")
print("*************** Signal 1 ***************")
for i,thresh_perecntage in enumerate(thresholds_percentage):
  size = sum([lst[i].nbytes for lst in wav_coff_thresh_1])
  print("Thresholding of", thresh_perecntage, "==>", size, "bytes")
  old_sizes_1.append(size)

print("\n*************** Signal 2 ***************")
for i,thresh_perecntage in enumerate(thresholds_percentage):
  size = sum([lst[i].nbytes for lst in wav_coff_thresh_2])
  print("Thresholding of", thresh_perecntage, "==>", size, "bytes")
  old_sizes_2.append(size)

Sizes after thresholding and before quantization:

*************** Signal 1 ***************
Thresholding of 0.05 ==> 112604224 bytes
Thresholding of 0.1 ==> 112604224 bytes
Thresholding of 0.15 ==> 112604224 bytes
Thresholding of 0.2 ==> 112604224 bytes

*************** Signal 2 ***************
Thresholding of 0.05 ==> 28151136 bytes
Thresholding of 0.1 ==> 28151136 bytes
Thresholding of 0.15 ==> 28151136 bytes
Thresholding of 0.2 ==> 28151136 bytes


After applying the threshold, quantize the coefficients. A simple way to do that is to store
them in a smaller word size. For example, you may decide to typecast the coefficients to
int8. Please note that you must first rescale your coefficients such that the maximum and
minimum coefficient are within a range that can be represented by int8 type.

In [14]:
##############################################################################################
###### THIS CODE WAS CREATED FOR ONLY 1 SIGNAL (JUST FOR EXPLANATION) ==> DON'T RUN IT  ######
##############################################################################################

quantized_coeffs = []

# Find the maximum and minimum values for each array in the list
for wav_coffy in wav_coff_thresh_1:
    quantized_coffy = []
    for arr in wav_coffy:
        max_value = np.max(arr)
        min_value = np.min(arr)
        
        # Rescale the coefficients to fit within the desired range
        new_min = -32768
        new_max = 32767
        rescaled_coeffs = (arr - min_value) / (max_value - min_value) * (new_max - new_min) + new_min
        
        # Typecast the rescaled coefficients to int16
        quantized_arr = rescaled_coeffs.astype(np.int16)
        
        # Store the quantized coefficients in the list
        quantized_coffy.append(quantized_arr)
    quantized_coeffs.append(quantized_coffy)

In [15]:
quantized_coeffs_1 = []
quantized_coeffs_2 = []


# Find the maximum and minimum values for each array in the list
for wav_coffy_1, wav_coffy_2 in zip(wav_coff_thresh_1, wav_coff_thresh_2):
    quantized_coffy_1 = []
    quantized_coffy_2 = []
    for arr_1, arr_2 in zip(wav_coffy_1, wav_coffy_2):
        max_value_1 = np.max(arr_1)
        min_value_1 = np.min(arr_1)
        max_value_2 = np.max(arr_2)
        min_value_2 = np.min(arr_2)

        # Rescale the coefficients to fit within the desired range
        new_min = -32768
        new_max = 32767
        rescaled_coeffs_1 = (arr_1 - min_value_1) / (max_value_1 - min_value_1) * (new_max - new_min) + new_min
        rescaled_coeffs_2 = (arr_2 - min_value_2) / (max_value_2 - min_value_2) * (new_max - new_min) + new_min

        # Typecast the rescaled coefficients to int16
        quantized_arr_1 = rescaled_coeffs_1.astype(np.int16)
        quantized_arr_2 = rescaled_coeffs_2.astype(np.int16)

        # Store the quantized coefficients in the list
        quantized_coffy_1.append(quantized_arr_1)
        quantized_coffy_2.append(quantized_arr_2)

    quantized_coeffs_1.append(quantized_coffy_1)
    quantized_coeffs_2.append(quantized_coffy_2)

Find the number of bytes occupied by the coefficients of one image. This should already be
smaller than the original size due to the quantization step. However, note that we haven’t
compressed the image yet.

In [16]:
new_sizes_1 = []
new_sizes_2 = []

print("Sizes after thresholding and after quantization:\n")

print("*************** Signal 1 ***************")
for i,thresh_perecntage in enumerate(thresholds_percentage):
  size = sum([lst[i].nbytes for lst in quantized_coeffs_1])
  print("Thresholding of", thresh_perecntage, "==>", size, "bytes")
  new_sizes_1.append(size)

print("\n*************** Signal 2 ***************")
for i,thresh_perecntage in enumerate(thresholds_percentage):
  size = sum([lst[i].nbytes for lst in quantized_coeffs_2])
  print("Thresholding of", thresh_perecntage, "==>", size, "bytes")
  new_sizes_2.append(size)

Sizes after thresholding and after quantization:

*************** Signal 1 ***************
Thresholding of 0.05 ==> 28151056 bytes
Thresholding of 0.1 ==> 28151056 bytes
Thresholding of 0.15 ==> 28151056 bytes
Thresholding of 0.2 ==> 28151056 bytes

*************** Signal 2 ***************
Thresholding of 0.05 ==> 7037784 bytes
Thresholding of 0.1 ==> 7037784 bytes
Thresholding of 0.15 ==> 7037784 bytes
Thresholding of 0.2 ==> 7037784 bytes


In [17]:
print("Size reduction after quantization:\n")

print("********** Signal 1 **********")
for old, new, thresh in zip(old_sizes_1, new_sizes_1, thresholds_percentage):
  print("Thresholding of ", thresh, " ==>", old/new)

print("\n********** Signal 2 **********")
for old, new, thresh in zip(old_sizes_1, new_sizes_1, thresholds_percentage):
  print("Thresholding of ", thresh, " ==>", old/new)

Size reduction after quantization:

********** Signal 1 **********
Thresholding of  0.05  ==> 4.0
Thresholding of  0.1  ==> 4.0
Thresholding of  0.15  ==> 4.0
Thresholding of  0.2  ==> 4.0

********** Signal 2 **********
Thresholding of  0.05  ==> 4.0
Thresholding of  0.1  ==> 4.0
Thresholding of  0.15  ==> 4.0
Thresholding of  0.2  ==> 4.0


**`Done`**